## Source: https://dashee87.github.io/football/python/predicting-football-results-with-statistical-modelling/

In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam

In [3]:
df = pd.read_csv('./results/results.csv')

In [5]:
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [11]:
epl_1617 = df[['home_team', 'away_team', 'home_score', 'away_score']]
epl_1617 = epl_1617.rename(columns={'home_score': 'HomeGoals', 'away_score': 'AwayGoals',
                                    'home_team': 'HomeTeam', 'away_team': "AwayTeam"})
epl_1617.head()

,HomeTeam,AwayTeam,HomeGoals,AwayGoals
0,Scotland,England,0,0
1,England,Scotland,4,2
2,Scotland,England,2,1
3,England,Scotland,2,2
4,Scotland,England,3,0


In [16]:
epl_1617.tail()

,HomeTeam,AwayTeam,HomeGoals,AwayGoals
43737,England,Hungary,0,4
43738,Germany,Italy,5,2
43739,Netherlands,Wales,3,2
43740,Poland,Belgium,0,1
43741,Armenia,Scotland,1,4


In [12]:
epl_1617 = epl_1617[:-10]
epl_1617.mean()


/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_48650/1701789677.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  epl_1617.mean()


HomeGoals    1.740204
AwayGoals    1.180056
dtype: float64

In [13]:
# probability of draw between home and away team
skellam.pmf(0.0,  epl_1617.mean()[0],  epl_1617.mean()[1])


/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_48650/2555191340.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  skellam.pmf(0.0,  epl_1617.mean()[0],  epl_1617.mean()[1])


0.23627079995825298

In [14]:
# probability of home team winning by one goal
skellam.pmf(1,  epl_1617.mean()[0],  epl_1617.mean()[1])


/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_48650/3395712009.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  skellam.pmf(1,  epl_1617.mean()[0],  epl_1617.mean()[1])


0.2294120693131687

In [15]:
# importing the tools required for the Poisson regression model
import statsmodels.api as sm
import statsmodels.formula.api as smf

goal_model_data = pd.concat([epl_1617[['HomeTeam','AwayTeam','HomeGoals']].assign(home=1).rename(
            columns={'HomeTeam':'team', 'AwayTeam':'opponent','HomeGoals':'goals'}),
           epl_1617[['AwayTeam','HomeTeam','AwayGoals']].assign(home=0).rename(
            columns={'AwayTeam':'team', 'HomeTeam':'opponent','AwayGoals':'goals'})])

poisson_model = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                87484
Model:                            GLM   Df Residuals:                    86864
Model Family:                 Poisson   Df Model:                          619
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.3085e+05
Date:                Fri, 02 Sep 2022   Deviance:                   1.0994e+05
Time:                        13:13:15   Pearson chi2:                 1.04e+05
No. Iterations:                    22   Pseudo R-squ. (CS):             0.3360
Covariance Type:            nonrobust                                         
================================================================================================================
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                       -0.5560      0.260     -2.137      0.033      -1.066      -0.046
team[T.Afghanistan]                             -0.3551      0.204     -1.743      0.081      -0.754       0.044
team[T.Albania]                                  0.5034      0.191      2.637      0.008       0.129       0.878
team[T.Alderney]                                -1.3790      0.325     -4.246      0.000      -2.016      -0.743
team[T.Algeria]                                  0.7934      0.186      4.257      0.000       0.428       1.159
team[T.American Samoa]                          -1.5775      0.250     -6.316      0.000      -2.067      -1.088
team[T.Andalusia]                                1.0395      0.279      3.726      0.000       0.493       1.586
team[T.Andorra]                                 -0.6797      0.224     -3.037      0.002      -1.118      -0.241
team[T.Angola]                                   0.4251      0.189      2.244      0.025       0.054       0.796
team[T.Anguilla]                                -1.3695      0.246     -5.563      0.000      -1.852      -0.887
team[T.Antigua and Barbuda]                     -0.1014      0.193     -0.524      0.600      -0.480       0.278
team[T.Arameans Suryoye]                         0.1702      0.297      0.573      0.566      -0.412       0.752
team[T.Argentina]                                1.4282      0.184      7.744      0.000       1.067       1.790
team[T.Armenia]                                  0.5418      0.194      2.791      0.005       0.161       0.922
team[T.Artsakh]                                  0.3576      0.245      1.457      0.145      -0.123       0.838
team[T.Aruba]                                   -0.0876      0.200     -0.438      0.662      -0.480       0.305
team[T.Asturias]                                 1.8310      0.703      2.604      0.009       0.453       3.209
team[T.Australia]                                0.9009      0.185      4.867      0.000       0.538       1.264
team[T.Austria]                                  1.3060      0.185      7.070      0.000       0.944       1.668
team[T.Azerbaijan]                               0.1420      0.196      0.724      0.469      -0.243       0.527
team[T.Bahamas]                                 -0.8117      0.231     -3.518      0.000      -1.264      -0.359
team[T.Bahrain]                                  0.4357      0.187      2.330      0.020       0.069       0.802
team[T.Bangladesh]                              -0.6459      0.197     -3.286      0.001      -1.031      -0.261
team[T.Barawa]                                  -0.8553      0.306     -2.792      0.005      -1.456      -0.255
team[T.Barbados]                

- Dep. Variable: 	goals 	
- No. Observations: 	87484
- Model: 	GLM 	
- Df Residuals: 	86864
- Model Family: 	
- Poisson 	Df Model: 	619
- Link Function: 	Log 	
- Scale: 	1.0000
- Method: 	IRLS 	
- Log-Likelihood: 	-1.3085e+05
- Date: 	Fri, 02 Sep 2022 	
- Deviance: 	1.0994e+05
- Time: 	13:13:15 	
- Pearson chi2: 	1.04e+05
- No. Iterations: 	22 	
- Pseudo R-squ. (CS): 	0.3360
- Covariance Type: 	nonrobust 		

In [17]:
poisson_model.predict(pd.DataFrame(data={'team': 'Italy', 'opponent': 'Germany',
                                       'home':0},index=[1]))


1    1.08093
dtype: float64

In [18]:
poisson_model.predict(pd.DataFrame(data={'team': 'Germany', 'opponent': 'Italy',
                                       'home':1},index=[1]))


1    1.628253
dtype: float64

In [19]:
def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'team': homeTeam, 
                                                            'opponent': awayTeam,'home':1},
                                                      index=[1])).values[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'team': awayTeam, 
                                                            'opponent': homeTeam,'home':0},
                                                      index=[1])).values[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [home_goals_avg, away_goals_avg]]
    return(np.outer(np.array(team_pred[0]), np.array(team_pred[1])))
simulate_match(poisson_model, 'Germany', 'Italy', max_goals=3)


array([[0.06659117, 0.07198042, 0.03890291, 0.01401711],
       [0.10842727, 0.11720233, 0.06334378, 0.02282341],
       [0.0882735 , 0.09541751, 0.05156985, 0.01858114],
       [0.04791053, 0.05178795, 0.02798958, 0.01008493]])

In [22]:
ger_ita = simulate_match(poisson_model, "Germany", "Italy", max_goals=10)
# Germany win
np.sum(np.tril(ger_ita, -1))


0.5010611508389644

In [23]:
# draw
np.sum(np.diag(ger_ita))


0.24663969384970882

In [24]:
# Italy win
np.sum(np.triu(ger_ita, 1))

0.2522979222204328

# -----------

## Switzerland vs. Brazil

In [28]:
swi_bra = simulate_match(poisson_model, "Switzerland", "Brazil", max_goals=10)
# Switzerland win
np.sum(np.tril(swi_bra, -1))


0.20170188908064768

In [29]:
# draw
np.sum(np.diag(swi_bra))

0.21619218725298117

In [30]:
# brazil win
np.sum(np.triu(swi_bra, 1))

0.5820987276129258

## Switzerland vs Brazil
- swiss win - 20.17%
- draw - 21.6%
- Brazil win - 58.2%

## Switzerland vs. Serbia

In [31]:
swi_ser = simulate_match(poisson_model, "Switzerland", "Serbia", max_goals=10)
# Switzerland win
np.sum(np.tril(swi_ser, -1))


0.4028873863440675

In [32]:
# draw
np.sum(np.diag(swi_ser))

0.26006522449367003

In [33]:
# Serbia win
np.sum(np.triu(swi_ser, 1))

0.3370470141693518

- Switzerland win - 40.28%
- draw - 26%
- Serbia win - 33.7%

## Switzerland vs. Cameroon

In [34]:
swi_cam = simulate_match(poisson_model, "Switzerland", "Cameroon", max_goals=10)
# Switzerland win
np.sum(np.tril(swi_cam, -1))


0.6060946076738891

In [35]:
# draw
np.sum(np.diag(swi_cam))

0.22387930797268316

In [36]:
# Serbia win
np.sum(np.triu(swi_cam, 1))

0.17002228895712826

- Switzerland wins 60.6%
- Draw - 22.38%
- Cameroon wins - 17%

# --------------

### Switzerland vs Brazil
- swiss win - 20.17%
- draw - 21.6%
- Brazil win - 58.2%

### Switzerland vs. Serbia
- Switzerland win - 40.28%
- draw - 26%
- Serbia win - 33.7%

### Switzerland vs. Cameroon
- Switzerland wins 60.6%
- Draw - 22.38%
- Cameroon wins - 17%




Ecuador
USA

In [49]:
usa_ecu = simulate_match(poisson_model, "United States", "Ecuador", max_goals=10)
# usa win
np.sum(np.tril(usa_ecu, -1))


0.4650139479414346

In [50]:
# draw
np.sum(np.diag(usa_ecu))

0.25225447579592863

In [51]:
# Serbia win
np.sum(np.triu(usa_ecu, 1))

0.2827307821365378

## USA vs. Ecuador
- USA wins 46.5%
- Draw 25.22%
- Ecuador wins 28.27

In [60]:
usa_wal = simulate_match(poisson_model, "United States", "Wales", max_goals=10)
# usa win
np.sum(np.tril(usa_wal, -1))


0.42632852821158906

In [61]:
# draw
np.sum(np.diag(usa_wal))

0.2607851922892372

In [62]:
# wal win
np.sum(np.triu(usa_wal, 1))

0.31288588013168617

# Switzerland vs. Portugal

In [63]:
swi_por = simulate_match(poisson_model, "Switzerland", "Portugal", max_goals=10)
# switzerland win
np.sum(np.tril(swi_por, -1))


0.359680689211343

In [64]:
# draw
np.sum(np.diag(swi_por))

0.2535175980768691

In [65]:
# wal win
np.sum(np.triu(swi_por, 1))

0.3868011675075371

# Switzerland vs. Germany


In [82]:
swi_ger = simulate_match(poisson_model, "Switzerland", "Germany", max_goals=10)
# switzerland win
np.sum(np.tril(swi_ger, -1))


0.26118360305637545

In [83]:
# draw
np.sum(np.diag(swi_ger))

0.21986714756402367

In [84]:
# Ger win
np.sum(np.triu(swi_ger, 1))

0.5189424312481423

# Belgium vs. Germany

In [66]:
bel_ger = simulate_match(poisson_model, "Belgium", "Germany", max_goals=10)
# switzerland win
np.sum(np.tril(bel_ger, -1))


0.3375551560762905

In [67]:
# draw
np.sum(np.diag(bel_ger))

0.22363267805743073

In [68]:
# wal win
np.sum(np.triu(bel_ger, 1))

0.43880738153300514

# Belgium vs. Switzerland

In [69]:
swi_bel = simulate_match(poisson_model, "Switzerland", "Belgium", max_goals=10)
# switzerland win
np.sum(np.tril(swi_bel, -1))


0.42302893281259346

In [71]:
# draw
np.sum(np.diag(swi_bel))

0.22957146916279222

In [72]:
# Belgium win
np.sum(np.triu(swi_bel, 1))

0.34739657987244954

# Belgium vs. Brazil

In [76]:
bel_bra = simulate_match(poisson_model, "Belgium", "Brazil", max_goals=10)
# Belgium win
np.sum(np.tril(bel_bra, -1))


0.26435093095098783

In [77]:
# draw
np.sum(np.diag(bel_bra))

0.22701714787789398

In [78]:
# Brazil win
np.sum(np.triu(bel_bra, 1))

0.5086276935211699

# France vs. Argentina

In [73]:
fra_arg = simulate_match(poisson_model, "France", "Argentina", max_goals=10)
# France win
np.sum(np.tril(fra_arg, -1))


0.38484993335567635

In [74]:
# draw
np.sum(np.diag(fra_arg))

0.2512585728920014

In [75]:
# Argentina win
np.sum(np.triu(fra_arg, 1))

0.3638908625041707

# France vs. Brazil

In [79]:
fra_bra = simulate_match(poisson_model, "France", "Brazil", max_goals=10)
# France win
np.sum(np.tril(fra_bra, -1))


0.3164677633489853

In [80]:
# draw
np.sum(np.diag(fra_bra))

0.2462925218278481

In [81]:
# Brazil win
np.sum(np.triu(fra_bra, 1))

0.43723869469302723